### First notebook to play with the Google Maps API (Places)

In [43]:
import googlemaps
import requests
import json
import pandas as pd
import time

In [8]:
API_KEY = "XXX"

In [9]:
map_client = googlemaps.Client(API_KEY)

In [10]:
dir(map_client)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_generate_auth_url',
 '_get',
 '_get_body',
 '_request',
 'addressvalidation',
 'base_url',
 'channel',
 'clear_experience_id',
 'client_id',
 'client_secret',
 'directions',
 'distance_matrix',
 'elevation',
 'elevation_along_path',
 'find_place',
 'geocode',
 'geolocate',
 'get_experience_id',
 'key',
 'nearest_roads',
 'place',
 'places',
 'places_autocomplete',
 'places_autocomplete_query',
 'places_nearby',
 'places_photo',
 'queries_per_minute',
 'queries_per_second',
 'queries_quota',
 'requests_kwargs',
 'retry_over_query_limit',
 'retry_timeout',
 'reverse_geocode',
 'sent_times',
 'session',
 'set_experience_id

In [16]:
endpoint_url = "https://places.googleapis.com/v1/places:searchText"

In [40]:
headers = {
    'Content-Type': 'application/json',
    'X-Goog-Api-Key': API_KEY,
    'X-Goog-FieldMask': '*'  # Get all fields for now
}

area = "Schwabing-West"

payload = {
    "textQuery": f"Italian restaurants in {area}, Munich",
    "maxResultCount": 50,
    "locationBias": {
        "rectangle": {
            "low": {
                "latitude": 48.061,  # approx bounding box
                "longitude": 11.360
            },
            "high": {
                "latitude": 48.220,
                "longitude": 11.722
            }
        }
    }
}

In [44]:
# Function to make the API request and get data
def fetch_data(payload):
    response = requests.post(endpoint_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

# Store results in a list
results = []

# Make up to 5 requests
max_requests = 5
requests_made = 0

while len(results) < 50 and requests_made < max_requests:
    requests_made += 1
    
    # Fetch data
    data = fetch_data(payload)
    
    # Check if we have places in the response
    if data and 'places' in data:
        for place in data.get('places', []):
            results.append({
                "name": place.get('displayName', {}).get('text'),
                "address": place.get('formattedAddress'),
                "area": area,
                "userRatingCount": place.get('userRatingCount', 0),
                "placeId": place.get('id'),
            })
        
        # Check if there are more results to fetch
        if 'nextPageToken' in data:
            # Set the token for the next request
            payload["pageToken"] = data['nextPageToken']
            print(f"Fetching more restaurants... Request {requests_made}")
        else:
            break
    
    # Wait before making the next request to avoid hitting rate limits
    time.sleep(2)

# Print how many results were fetched
print(f"Total number of restaurants fetched: {len(results)}")

# Convert the results into a Pandas DataFrame
df = pd.DataFrame(results)

Fetching more restaurants... Request 1
Total number of restaurants fetched: 40


In [51]:
df

,name,address,area,userRatingCount,placeId
0,Trattoria Pizzeria Da Pino,"Hohenzollernstraße 26, 80801 München, Germany",Schwabing-West,413,ChIJXS1FOsd1nkcRBBlJleMzHvM
1,Pepe e Sale,"Wilhelmstraße 15, 80801 München, Germany",Schwabing-West,208,ChIJRXFJRMd1nkcR1XojL4vrofA
2,Trattoria Pizzeria Da Bello E Bello,"Elisabethstraße 19, 80796 München, Germany",Schwabing-West,507,ChIJvy51z9x1nkcR8US-x7MZBDA
3,Vini e Panini,"Nordendstraße 45, 80801 München, Germany",Schwabing-West,122,ChIJJ9mTvMN1nkcRw42y0VY-Mqc
4,Trattoria La Stella,"Hohenstaufenstraße 2, 80801 München, Germany",Schwabing-West,207,ChIJpaWZJ8J1nkcRuJUZqujpjW0
5,Passaparola Schwabing,"Kaiserstraße 47, 80801 München, Germany",Schwabing-West,418,ChIJ--v4m8Z1nkcRP9-FCgXADgY
6,Royal Cookhouse Pizzeria,"Belgradstraße 47, 80796 München, Germany",Schwabing-West,493,ChIJ62y0YiV1nkcRvXCclJzO5ys
7,Bucci Bar,"Occamstraße 9, 80802 München, Germany",Schwabing-West,127,ChIJM4DTvYp1nkcRvIdPoVb78JI
8,Osteria Blu Notte,"Schleißheimer Str. 69, 80797 München, Germany",Schwabing-West,230,ChIJxx_QFt51nkcR-GOwtsvG1I0
9,Trattoria La Piazza,"Kölner Pl. 7, 80804 München, Germany",Schwabing-West,834,ChIJXTcUK8x1nkcRjIepGhhp_f8


In [52]:
df.to_csv('/Users/adolci/OneDrive - Axel Springer SE/Python/private/wine_selling/restaurants_in_schwabing_west.csv', index=False)